# Dynamic Programming and Optimal Control
## Problem Set 3, Problem 5

**Python script that solves Problem 5 of Problem Set 3 by applying applying the Label Correcting method and A* algorithm.** 

**We use [NumPy](https://numpy.org/) and [SciPy](https://scipy.org/) packages. You can install these packages using `pip install` or `conda install` command depending on your package manager. You can also find the installation guide in the package websites or documentations.**

In [ ]:
import scipy.io
import numpy as np
import time

### Label correcting algorithm function

In [ ]:
def lca(A, start_node, terminal_node):
    # Executes Label Correcting algorithm (Book Dynamic Programming and Optimal
    # Control, Bertsekes, page 81) using the depth-first method.

    # Input:
    #   A               [NxN] matrix, where the element A(i,j) = a_ij is the cost
    #                   to move from node i to j.
    #   start_node      Start node of desired shortest path, scalar from 1 to N.
    #   terminal_node   Terminal node of desired shortest path, scalar from 1
    #                   to N.

    # Output:
    #   opt_cost        Cost of the shortest path(s), scalar:
    #   opt_path        Row vector containing the shortest path, e.g.
    #                   opt_path = [0 32 44 43 78 99].

    N = len(A)  # Dimension of the problem: N = total number of nodes
    d = np.ones(N) * np.inf  # Vector holding label d for each node. d(i) represents
    # the shortest path found so far from start node to i.
    d[start_node] = 0
    parent = np.ones(N) * np.inf  # Vector containing the parent of the shortest path
    # found so far for each node.
    parent[start_node] = -1
    OPEN = np.zeros(N)  # List cotaining all the nodes that are currently
    # active in the sense that they are candidates for
    # further examination (candidates list).
    pointer_OPEN = 1  # Pointer which always points to the last element in OPEN.
    OPEN[pointer_OPEN] = start_node
    UPPER = np.inf  # Label dt, representing the shortest path to the end found so far.

    # Check start and terminal node
    # Make sure that the start and terminal node are valid.
    if start_node == terminal_node:
        opt_cost = 0
        opt_path = [start_node, terminal_node]
        return opt_cost, opt_path  # Done.

    if (start_node >= N or terminal_node >= N) or (start_node < 0 or terminal_node < 0):
        opt_cost = np.inf
        opt_path = None
        return opt_cost, opt_path  # Done.

    # Execute algorithm
    while 1:
        # STEP 1: Remove a node i from OPEN and for each child j of i, execute STEP 2.
        i = int(OPEN[pointer_OPEN])
        OPEN[pointer_OPEN] = 0
        pointer_OPEN = pointer_OPEN - 1

        children = np.where(A[i, :] != np.inf)
        children = children[0]
        if i in children:
            children = np.delete(children, np.where(children == i))

        for j in children:
            # STEP 2: If d_i + a_ij < min(d_j,UPPER), set d_j = d_i + a_ij and
            # set i to be the parent of j.
            if d[i] + A[i, j] < min([d[j], UPPER]):
                d[j] = d[i] + A[i, j]

                parent[j] = int(i)

                # In addition, if j != t, place j in OPEN if it is not already
                # in OPEN, while if j == t, set UPPER to the new value d_i +
                # a_it of d_t
                if j != terminal_node:
                    if not (j in OPEN):
                        pointer_OPEN += 1
                        OPEN[pointer_OPEN] = j
                else:
                    UPPER = d[j]

        # STEP 3: If OPEN is empty, terminate; else go to STEP 1.
        if not pointer_OPEN:
            break

    # UPPER is equal to the cost of the shortest path.
    opt_cost = UPPER

    # Construct shortest path
    # Start at terminal node and, for each node, take its parent node until we
    # find ourselves at the start node.
    opt_path = [terminal_node]
    while opt_path[-1] != start_node:
        opt_path.append(int(parent[int(opt_path[-1])]))
    opt_path.reverse()  # Reverse path: start_node -> terminal_node
    return opt_cost, opt_path

### A* algorithm function

In [ ]:
def astar(A, start_node, terminal_node):
    # [opt_cost, opt_path] = lca(A,start_node,terminal_node)

    # Executes A* algorithm (Book Dynamic Programming and Optimal
    # Control, Bertsekes, page 87) using the depth-first method.

    # Input:
    #   A               [NxN] matrix, where the element A(i,j) = a_ij is the cost
    #                   to move from node i to j.
    #   start_node       Start node of desired shortest path, scalar from 1 to N.
    #   terminal_node    Terminal node of desired shortest path, scalar from 1
    #                   to N.

    # Output:
    #   opt_cost         Cost of the shortest path(s), scalar:
    #   opt_path         Row vector containing the shortest path, e.g.
    #                   opt_path = [0 33 45 43 79 99].

    # Initialization
    N = len(A)  # Dimension of the problem: N = total number of nodes
    d = np.ones(N) * np.inf  # Vector holding label d for each node. d(i) represents
    # the shortest path found so far from start node to i.
    d[start_node] = 0
    parent = np.ones(N) * np.inf  # Vector containing the parent of the shortest path
    # found so far for each node.
    parent[start_node] = 0
    OPEN = np.zeros(N)  # List cotaining all the nodes that are currently
    # active in the sense that they are candidates for
    # further examination (candidates list).
    pointer_OPEN = 1  # Pointer which always points to the last element in OPEN.
    OPEN[pointer_OPEN] = start_node
    UPPER = np.inf  # Label dt, representing the shortest path to the end found so far.

    # Check start and terminal node
    # Make sure that the start and terminal node are valid.
    if start_node == terminal_node:
        opt_cost = 0
        opt_path = [start_node, terminal_node]
        return opt_cost, opt_path  # Done.

    if (start_node >= N or terminal_node >= N) or (start_node < 0 or terminal_node < 0):
        opt_cost = np.inf
        opt_path = None
        return opt_cost, opt_path  # Done.

    # Execute algorithm
    while 1:
        # STEP 1: Remove a node i from OPEN and for each child j of i, execute STEP 2.
        i = int(OPEN[pointer_OPEN])
        OPEN[pointer_OPEN] = 0
        pointer_OPEN = pointer_OPEN - 1

        children = np.where(A[i, :] != np.inf)
        children = children[0]
        if i in children:
            children = np.delete(children, np.where(children == i))

        for j in children:
            # STEP 2: If d_i + a_ij < and  d_i + a_ij + h_j < UPPER,
            # set d_j = d_i + a_ij and set i to be the parent of j.
            if (d[i] + A[i, j] < d[j] and 
                d[i] + A[i, j] + abs(j - terminal_node) < UPPER):
                d[j] = d[i] + A[i, j]
                parent[j] = i

                # In addition, if j ~= t, place j in OPEN if it is not already
                # in OPEN, while if j == t, set UPPER to the new value d_i +
                # a_it of d_t
                if j != terminal_node:
                    if not (j in OPEN):
                        pointer_OPEN += 1
                        OPEN[pointer_OPEN] = j
                else:
                    UPPER = d[j]

        #  STEP 3: If OPEN is empty, terminate; else go to STEP 1.
        if not pointer_OPEN:
            break

    # DONE.
    # UPPER is equal to the cost of the shortest path.
    opt_cost = UPPER
    # Construct shortest path
    # Start at terminal node and, for each node, take its parent node until we
    # find ourselves at the start node.
    opt_path = [terminal_node]
    while opt_path[-1] != start_node:
        opt_path.append(int(parent[int(opt_path[-1])]))
    opt_path.reverse()  # Reverse path: start_node -> terminal_node
    return opt_cost, opt_path

### Initialize variables

In [ ]:
mat = scipy.io.loadmat("A.mat")
# Load matrix A that contains all the transition costs A(i,j) = a_ij to get from i to j.
A = mat["A"]  
N = len(A)  # Dimension of the problem: N = total number of nodes

### Define start and terminal node

In [ ]:
# Default values:
#   start_node = 0
#   terminal_node = 99

# Minimum path length (minimum total cost): 100
# Path: 0 -> 2 -> 40 -> 50 -> 99

start_node = 0
terminal_node = 99

### Label Correcting Algorithm
Solve shortest path problem using the Label Correcting Algorithm

In [ ]:
t = time.time()
# Your implementation of the Label Correcting aglorithm.
[opt_cost_1, opt_path_1] = lca(A, start_node, terminal_node)
time1 = time.time() - t

### A* Algorithm
Solve shortest path problem using the A* Algorithm

In [ ]:
t = time.time()
# Your implementation of the A* algorithm.
[opt_cost_2, opt_path_2] = astar(A, start_node, terminal_node)  
time2 = time.time() - t

### Print results

In [ ]:
print("Results")
print("Problem with ", N, " nodes.")
print("Optimal path from node ", start_node, " to ", terminal_node, ":")
print("\033[1mLabel Correcting Algorithm\033[0m")
print("Execution time: ", time1, " s.")
print("Minimum path length (minimum total cost): ", opt_cost_1)
print("Path: ", [n + 1 for n in opt_path_1])
print("\033[1mA* Algorithm\033[0m")
print(
    "Execution time: ", time2, "s  (", time2 / time1, " times the time for method 1)."
)
print("Minimum path length (minimum total cost): ", opt_cost_2)
print("Path: ", [n + 1 for n in opt_path_2])